In [57]:
import numpy as np
import numpy.random as random
import pandas as pd
from pandas import Series, DataFrame
from datetime import datetime, timedelta
import os
import re

In [69]:
# 基本的に以下を行えば準備完了

# 前処理を行うファイルを./data/xxxxxxxxtestx/InsoleSensor/0_defaultに用意する
# 前処理を行うファイルを./data/xxxxxxxxtestx/Opti-track/0_defaultに用意する
# ここでパスを指定する
BASE_DIR = './data/20250518test4'

## P2P-Insole前処理用

In [59]:
# データクリーナー1
# -方向データの削除
# -データを0.01秒刻みに整形

data_path = BASE_DIR + '/InsoleSensor/0_default'
output_path = os.path.join(data_path, '../1_data_clean1')

# 出力先ディレクトリが存在しない場合は作成
os.makedirs(output_path, exist_ok=True)

csv_files = [f for f in os.listdir(data_path) if f.endswith('.csv')]

def resample_and_interpolate(data):
    # タイムスタンプ列の名前を明示的に指定
    timestamp_col = 'Timestamp'
    if timestamp_col not in data.columns:
        print(f"エラー: タイムスタンプ列 '{timestamp_col}' がデータに存在しません。")
        return None

    # タイムスタンプをdatetimeオブジェクトに変換し、インデックスに設定
    # 数値（秒）として保存されていると仮定し、基準日（エポック）からの経過時間として処理
    data[timestamp_col] = pd.to_datetime(data[timestamp_col], unit='s', origin='unix')
    data = data.set_index(timestamp_col)

    # 重複したインデックスを処理 (最初の出現を保持)
    data = data[~data.index.duplicated(keep='first')]

    # 0.01秒刻みでリサンプリングし、欠損値を線形補間
    data_resampled = data.resample('10ms').mean().interpolate(method='linear')

    return data_resampled

for csv_file in csv_files:
    # CSVファイルの読み込み(header=1 <=> 一行目の//DN:108,SN:35の行を削除)
    file_path = os.path.join(data_path, csv_file)
    data = pd.read_csv(file_path, header=1)

    # 方位データを削除
    if all(col in data.columns for col in ["Mag_x", "Mag_y", "Mag_z"]):
        data = data.drop(["Mag_x", "Mag_y", "Mag_z"], axis=1)

    # リサンプリング処理の適用
    data_resample = resample_and_interpolate(data.copy()) # 元のデータを変更しないようにコピーを渡す

    if data_resample is not None:
        # 出力ファイルの保存
        output_file_path = os.path.join(output_path, csv_file)
        data_resample.to_csv(output_file_path)

        # 処理内容の確認（例として最初の数行を表示）
        print(f'Processed {csv_file}:')
        print(data_resample.head())
    else:
        print(f'Error processing {csv_file}. スキップします。')

Processed 20241115_152500_left.csv:
                               P1   P2   P3        P4        P5        P6  \
Timestamp                                                                   
2024-11-15 06:20:02.540  0.023772  0.0  0.0  0.050205  0.534279  5.871815   
2024-11-15 06:20:02.550  0.023772  0.0  0.0  0.048463  0.528049  5.830763   
2024-11-15 06:20:02.560  0.023772  0.0  0.0  0.051990  0.528049  5.871815   
2024-11-15 06:20:02.570  0.022805  0.0  0.0  0.050205  0.534279  5.871815   
2024-11-15 06:20:02.580  0.023772  0.0  0.0  0.045108  0.528049  5.830763   

                               P7        P8        P9       P10  ...  \
Timestamp                                                        ...   
2024-11-15 06:20:02.540  4.976448  4.068412  3.355657  4.416726  ...   
2024-11-15 06:20:02.550  4.945159  4.129462  3.355657  4.416726  ...   
2024-11-15 06:20:02.560  4.976448  4.129462  3.355657  4.416726  ...   
2024-11-15 06:20:02.570  4.913990  4.129462  3.355657  4.416726 

In [60]:
# データクリーナー2
# -左右のタイムスタンプを一致させる

# フォルダのパスをここに指定してください
data_path = BASE_DIR + '/InsoleSensor/1_data_clean1' 
output_path = os.path.normpath(os.path.join(data_path, "../2_data_clean2"))
os.makedirs(output_path, exist_ok=True)

def sync_csv_by_timestamp(data_path, output_path):
    all_files = os.listdir(data_path)
    left_files = [f for f in all_files if '_left.csv' in f]

    for left_file in left_files:
        match = re.match(r'(.+)_left\.csv', left_file)
        if match:
            base_name = match.group(1)
            right_file = f"{base_name}_right.csv"

            if right_file in all_files:
                left_path = os.path.join(data_path, left_file)
                right_path = os.path.join(data_path, right_file)

                try:
                    left_df = pd.read_csv(left_path)
                    right_df = pd.read_csv(right_path)

                    if 'Timestamp' not in left_df.columns or 'Timestamp' not in right_df.columns:
                        print(f"'{left_file}' または '{right_file}' に 'Timestamp' カラムが存在しません。")
                        continue

                    left_df.set_index('Timestamp', inplace=True)
                    right_df.set_index('Timestamp', inplace=True)

                    common_timestamps = left_df.index.intersection(right_df.index)

                    if common_timestamps.empty:
                        print(f"'{left_file}' と '{right_file}' に共通のタイムスタンプがありません。")
                        continue

                    synced_left_df = left_df.loc[common_timestamps]
                    synced_right_df = right_df.loc[common_timestamps]

                    synced_left_path = os.path.join(output_path, f"{base_name}_left.csv")
                    synced_right_path = os.path.join(output_path, f"{base_name}_right.csv")

                    synced_left_df.to_csv(synced_left_path)
                    synced_right_df.to_csv(synced_right_path)

                    print(f"'{left_file}' と '{right_file}' を同期し、'{synced_left_path}' と '{synced_right_path}' に保存しました。")

                except Exception as e:
                    print(f"エラー: '{left_file}' と '{right_file}' の処理中にエラーが発生しました: {e}")
    
sync_csv_by_timestamp(data_path, output_path)
print("処理が完了しました。")

'20241115_152500_left.csv' と '20241115_152500_right.csv' を同期し、'data\20250518test3\InsoleSensor\2_data_clean2\20241115_152500_left.csv' と 'data\20250518test3\InsoleSensor\2_data_clean2\20241115_152500_right.csv' に保存しました。
'20241115_153100_left.csv' と '20241115_153100_right.csv' を同期し、'data\20250518test3\InsoleSensor\2_data_clean2\20241115_153100_left.csv' と 'data\20250518test3\InsoleSensor\2_data_clean2\20241115_153100_right.csv' に保存しました。
'20241115_153700_left.csv' と '20241115_153700_right.csv' を同期し、'data\20250518test3\InsoleSensor\2_data_clean2\20241115_153700_left.csv' と 'data\20250518test3\InsoleSensor\2_data_clean2\20241115_153700_right.csv' に保存しました。
'20241115_154300_left.csv' と '20241115_154300_right.csv' を同期し、'data\20250518test3\InsoleSensor\2_data_clean2\20241115_154300_left.csv' と 'data\20250518test3\InsoleSensor\2_data_clean2\20241115_154300_right.csv' に保存しました。
'20241115_154900_left.csv' と '20241115_154900_right.csv' を同期し、'data\20250518test3\InsoleSensor\2_data_clean2\20241115_15

In [61]:
# P2P-insoleのタイムスタンプがUTC(協定世界時)に合わせられている場合はこのコードを実行する

# フォルダのパスをここに指定してください
data_path = BASE_DIR + '/InsoleSensor/2_data_clean2' 
output_path = BASE_DIR + '/InsoleSensor/2_data_clean2' 

csv_files = [f for f in os.listdir(data_path) if f.endswith('.csv')]

def convert_utc_to_jst(csv_file_path):
    # CSVファイルを読み込む
    df = pd.read_csv(csv_file_path)

    # タイムスタンプ列をdatetime型に変換 (UTCとして認識)
    df['Timestamp'] = pd.to_datetime(df['Timestamp'], utc=True)

    # UTCからJSTに変換 (+9時間)
    df['Timestamp'] = df['Timestamp'].dt.tz_convert('Asia/Tokyo')

    return df
    

for csv_file in csv_files:
    # CSVファイルの読み込み
    file_path = os.path.join(data_path, csv_file)
    data = pd.read_csv(file_path)

    # UTCからJSTに変換
    converted_df = convert_utc_to_jst(file_path)
   
    # 出力ファイルの保存
    output_file_path = os.path.join(output_path, csv_file)
    converted_df.to_csv(output_file_path, index=False)   


## Opti-Track前処理用

In [62]:
# データクリーナー1
# -Frame, 偶数ポイント行の削除
# -データ頭の詳細情報を削除
# -ボーン以外のデータを削除するコード(X_42以降)

# 0_defaultフォルダに生データを入れる
data_path = BASE_DIR + '/Opti-track/0_default'
output_path = os.path.normpath(os.path.join(data_path, "../1_data_clean1"))
os.makedirs(output_path, exist_ok=True)

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(data_path) if f.endswith('.csv')]

# CSVファイルごとに処理を行う
for csv_file in csv_files:
    # CSVファイルを読み込み(エラーが出た場合はheaderを調整)
    file_path = os.path.join(data_path, csv_file)
    data = pd.read_csv(file_path, header = 6)

    display(data.head())

    data = data.drop(['Frame','X','Y','Z'],axis=1)
    for n in range(2, 41, 2):
        data = data.drop(['X.'+ str(n), 'Y.'+ str(n), 'Z.'+ str(n)], axis=1)

    for n in range(42, 500, 1):
        try:
            data = data.drop(['X.'+ str(n), 'Y.'+ str(n), 'Z.'+ str(n)], axis=1)
        except KeyError :
            print(['X.'+ str(n), 'Y.'+ str(n), 'Z.'+ str(n)], "not found in axis")
            break
    
    data.to_csv(output_path + "/" +csv_file, index=False)

,Frame,Time (Seconds),X,Y,Z,X.1,Y.1,Z.1,X.2,Y.2,...,Z.120,X.121,Y.121,Z.121,X.122,Y.122,Z.122,X.123,Y.123,Z.123
0,0,0.00,-0.892402,-0.295710,0.005703,-0.498586,871.353760,-46.918076,-5.460565,-0.147451,...,-111.838547,152.495941,908.782593,28.128002,-118.049644,993.266785,-126.481636,-162.903503,919.322754,-0.142413
1,1,0.01,-1.686048,-1.501143,0.645752,-5.050343,870.118408,-54.696842,-8.263016,-1.212070,...,-111.870689,152.451645,908.802307,28.111132,-118.087982,993.266052,-126.510948,-162.944412,919.323669,-0.176844
2,2,0.02,-2.127119,-2.847631,0.824781,-3.512707,868.961731,-53.804890,-10.627911,-2.160597,...,-111.899017,152.406693,908.820312,28.093828,-118.125778,993.265930,-126.541702,-162.986572,919.325806,-0.212116
3,3,0.03,-2.172044,-4.161436,0.762468,-1.253205,867.050659,-50.225803,-12.078902,-3.110315,...,-111.922325,152.360611,908.836365,28.076847,-118.162506,993.266602,-126.574486,-163.029480,919.329651,-0.248916
4,4,0.04,-1.964449,-5.151487,0.653666,-0.450463,864.590149,-48.565468,-12.800073,-3.853965,...,-111.939857,152.313049,908.850891,28.061701,-118.197952,993.268372,-126.609459,-163.072693,919.334656,-0.287946


['X.124', 'Y.124', 'Z.124'] not found in axis


,Frame,Time (Seconds),X,Y,Z,X.1,Y.1,Z.1,X.2,Y.2,...,Z.120,X.121,Y.121,Z.121,X.122,Y.122,Z.122,X.123,Y.123,Z.123
0,0,0.00,-1.263682,-0.473662,0.276909,-10.338716,871.857239,-49.498924,-6.339119,-0.383537,...,-110.475830,143.429123,909.173523,23.134998,-126.494736,995.804749,-129.743484,-171.192184,920.305725,-9.557451
1,1,0.01,-2.006080,-2.335066,0.495247,-11.416638,870.505432,-57.805302,-8.784081,-2.628509,...,-110.504410,143.447937,909.179382,23.107296,-126.503555,995.814392,-129.732346,-171.180847,920.305603,-9.546223
2,2,0.02,-2.442192,-3.893363,0.471386,-9.682819,869.358765,-57.642765,-11.189780,-4.014058,...,-110.534393,143.467194,909.184570,23.077688,-126.512703,995.824097,-129.722488,-171.169708,920.305298,-9.535896
3,3,0.03,-2.441602,-4.976621,0.308637,-8.679569,867.440979,-54.075249,-12.801524,-4.457030,...,-110.566521,143.487442,909.189209,23.044891,-126.522186,995.833557,-129.714279,-171.158813,920.304810,-9.527079
4,4,0.04,-2.140973,-5.763481,0.206976,-9.565400,864.910950,-51.941204,-13.642390,-4.650467,...,-110.601326,143.509003,909.193970,23.009478,-126.531960,995.842529,-129.707870,-171.148407,920.304321,-9.519944


['X.124', 'Y.124', 'Z.124'] not found in axis


,Frame,Time (Seconds),X,Y,Z,X.1,Y.1,Z.1,X.2,Y.2,...,Z.125,X.126,Y.126,Z.126,X.127,Y.127,Z.127,X.128,Y.128,Z.128
0,0,0.00,-1.344247,-0.574761,-0.215904,3.224892,870.644653,-29.888142,-6.137048,-0.491145,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0.01,-1.822448,-2.572242,0.003807,2.235324,868.586487,-31.445082,-7.468238,-2.990989,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,0.02,-2.249439,-4.008212,-0.099419,3.060926,867.090698,-27.728338,-9.588195,-4.120322,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,0.03,-2.451726,-5.074874,-0.311997,3.120261,864.997253,-24.063931,-11.242334,-4.514317,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,0.04,-2.450366,-5.872270,-0.456807,1.771444,862.523560,-22.585114,-12.249215,-4.764765,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


...

['X.127', 'Y.127', 'Z.127'] not found in axis


,Frame,Time (Seconds),X,Y,Z,X.1,Y.1,Z.1,X.2,Y.2,...,Z.128,X.129,Y.129,Z.129,X.130,Y.130,Z.130,X.131,Y.131,Z.131
0,0,0.00,-1.412736,-0.402362,-0.299226,7.619492,871.329773,-56.689999,-6.798099,-0.189671,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0.01,-2.298907,-2.096853,0.013955,5.910054,870.373901,-66.477982,-9.667384,-1.935714,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,0.02,-2.799525,-3.755209,0.053347,7.642736,869.258057,-66.939613,-12.168916,-3.226409,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,0.03,-2.800704,-5.101085,-0.063605,8.999472,867.275085,-63.367672,-13.707144,-3.960156,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,0.04,-2.472580,-6.106148,-0.149100,8.657993,864.708557,-61.304825,-14.400400,-4.452399,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


['X.132', 'Y.132', 'Z.132'] not found in axis


,Frame,Time (Seconds),X,Y,Z,X.1,Y.1,Z.1,X.2,Y.2,...,Z.125,X.126,Y.126,Z.126,X.127,Y.127,Z.127,X.128,Y.128,Z.128
0,0,0.00,-0.902660,-0.744365,-0.375033,27.756575,871.192444,-49.175552,-5.683951,-0.745622,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0.01,-1.761203,-2.554095,-0.097389,30.008688,871.530029,-57.216763,-8.785872,-3.114363,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,0.02,-2.167670,-3.748011,-0.060772,34.786366,870.898987,-56.008846,-11.278440,-4.271109,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,0.03,-2.105242,-4.647998,-0.167161,37.580593,869.051819,-52.380322,-12.738095,-4.825294,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,0.04,-1.759368,-5.337743,-0.290410,37.571491,866.592407,-50.988235,-13.396652,-5.140187,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


['X.129', 'Y.129', 'Z.129'] not found in axis


In [70]:
# データクリーナー2
# タイムスタンプを作成するコード
data_path = BASE_DIR + '/Opti-track/1_data_clean1'
output_path = os.path.normpath(os.path.join(data_path, "../2_data_clean2"))
os.makedirs(output_path, exist_ok=True)

def convert_time_seconds_to_timestamp(filename):
    try:
        # CSVファイルを読み込む
        df = pd.read_csv(filename)
    except FileNotFoundError:
        print(f"エラー: ファイル '{filename}' が見つかりません。")
        return None
    except Exception as e:
        print(f"エラー: CSVファイルの読み込み中にエラーが発生しました: {e}")
        return None

    try:
        # 正規表現でファイル名から日時とAM/PMを抽出
        base_filename = os.path.basename(filename)
        match = re.search(r'(\d{4}-\d{2}-\d{2})\s+(\d{2})\.(\d{2})\.(\d{2})\s+(AM|PM)', base_filename)
        if match:
            date_part = match.group(1).replace('-', '/')
            hour_part = int(match.group(2))
            minute_part = int(match.group(3))
            second_part = int(match.group(4))
            am_pm_part = match.group(5)
            start_datetime = datetime(int(date_part.split('/')[0]),
                                      int(date_part.split('/')[1]),
                                      int(date_part.split('/')[2]),
                                      hour_part, minute_part, second_part)
            if am_pm_part == 'PM' and hour_part != 12:
                start_datetime = start_datetime + timedelta(hours=12)
            elif am_pm_part == 'AM' and hour_part == 12:
                start_datetime = start_datetime - timedelta(hours=12) # midnight case

        else:
            print(f"エラー: ファイル '{base_filename}' から日時を抽出できませんでした。ファイル名が予期された形式ではありません。")
            return df

    except ValueError as e:
        print(f"エラー: ファイル '{base_filename}' から抽出した日時を解析できませんでした: {e}")
        return df  # ファイルは読み込めたので、変換前のDataFrameを返す

    # Time (Second) 列をtimedeltaオブジェクトに変換
    try:
        time_deltas = pd.to_timedelta(df['Time (Seconds)'], unit='s')
    except KeyError:
        print(f"エラー: ファイル '{base_filename}' に 'Time (Seconds)' という列が見つかりません。")
        return df  # 'Time (Second)' 列がない場合は、元のDataFrameを返す

    # 開始日時にtimedeltaを加算してTimestamp列を生成
    timestamp_column = start_datetime + time_deltas

    # データフレームにTimestamp列を追加
    df['Time (Seconds)'] = timestamp_column
    new_df = df.rename(columns={'Time (Seconds)' : 'Timestamp'})

    # タイムスタンプ列をdatetime型に変換
    new_df['Timestamp'] = pd.to_datetime(new_df['Timestamp'], utc=True)

    return new_df

def process_all_csv_in_folder(folder_path):
    if not os.path.isdir(folder_path):
        print(f"エラー: 指定されたパス '{folder_path}' はフォルダではありません。")
        return

    for filename in os.listdir(folder_path):
        if filename.endswith(".csv"):
            full_filepath = os.path.join(folder_path, filename)
            print(f"処理中のファイル: {full_filepath}")
            df_with_timestamp = convert_time_seconds_to_timestamp(full_filepath)
            if df_with_timestamp is not None:
                try:
                    df_with_timestamp.to_csv(os.path.join(output_path, filename), index=False)
                    print(f"処理後のファイルは '{output_path}' に保存されました。")
                except Exception as e:
                    print(f"エラー: 処理後のファイルの保存中にエラーが発生しました: {e}")
            print("-" * 30)


# フォルダ内のすべてのCSVファイルを処理
process_all_csv_in_folder(data_path)
print("処理が完了しました。")

処理中のファイル: ./data/20250518test4/Opti-track/1_data_clean1\Take 2024-12-12 03.06.59 PM.csv
処理後のファイルは 'data\20250518test4\Opti-track\2_data_clean2' に保存されました。
------------------------------
処理中のファイル: ./data/20250518test4/Opti-track/1_data_clean1\Take 2024-12-12 03.45.00 PM.csv
処理後のファイルは 'data\20250518test4\Opti-track\2_data_clean2' に保存されました。
------------------------------
処理中のファイル: ./data/20250518test4/Opti-track/1_data_clean1\Take 2024-12-12 04.28.00 PM.csv
処理後のファイルは 'data\20250518test4\Opti-track\2_data_clean2' に保存されました。
------------------------------
処理中のファイル: ./data/20250518test4/Opti-track/1_data_clean1\Take 2024-12-12 05.17.59 PM.csv
処理後のファイルは 'data\20250518test4\Opti-track\2_data_clean2' に保存されました。
------------------------------
処理が完了しました。


## P2P-insole, Optitrack共通処理

In [ ]:
# タイムスタンプ同期
# タイムスタンプ削除
# ファイル名をわかりやすいものに変換する
# /finalに保存する

In [ ]:
# 回転処理
# 位置処理

In [65]:
# nan値を変換
# スケーリング
# ガウスフィルター